- 충분하지 않은 데이터로 이미지 분류기를 훈련하라면 사전훈련된 모델의 하위층을 사용하는 것이 좋음

In [ ]:
import tensorflow as tf 
from tensorflow import keras
import tensorflow_datasets as tfds 

### Xception 모델을 사용해 꽃 이미지를 분류하는 모델


In [3]:
# 데이터 적재
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True) #with_info=True : 데이터셋 정보를 줌
dataset_size = info.splits["train"].num_examples # 3670
calss_names = info.features["label"].names # ["dendelion","daisy",...]
n_classes = info.features["label"].num_classes # 5

In [17]:
# train/test set 나누기 

# 처음 10%를 테스트셋, 다음 15% 검증셋, 남은 75% 훈련셋 
# test_split, valid_split, train_split = tfds.Split.TRAIN.subsplit([10, 15, 75])
# test_set = tfds.load("tf_flowers", split = valid_split, as_supervised=True)
# valid_set = tfds.load("tf_flowers", split = valid_split, as_supervised=True)
# train_set = tfds.load("tf_flowers", split = train_split, as_supervised=True)


test_set = tfds.load("tf_flowers", split = 'train[:10%]', as_supervised=True)
valid_set = tfds.load("tf_flowers", split = 'train[10%:25%]', as_supervised=True)
train_set = tfds.load("tf_flowers", split = 'train[25%:]', as_supervised=True)



In [11]:
# 이미지 전처리

# 224 X 224 크기로 조정 
def preprocess(image,label):
    resized_image = tf.image.resize(image,[224,224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [18]:
# 전처리 함수 3개를 데이터셋에 적용
# 배치크기를 지정 > 프리페치 적용 

batch_size = 32
train_size = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [19]:
# 데이터 증식을 추가하고 싶은 경우 : 훈련 이미지 랜덤하게 변경
# 1) 이미지를 랜덤하게 자르기 : tf.image.random_crop()
# 2) 이미지를 수평으로 랜덤하게 뒤집기 : tf.image.random_flip_left_right()


In [21]:
# Xception 모델 로드

base_model = keras.applications.xception.Xception(weights="imagenet", include_top = False)
                #include_Top=False : 네트워크 최상층에 해당하는 GlobalAveragePooling layer와 dense layer 제외
avg = keras.layers.GlobalAveragePooling2D()(base_model.output) #새로운 GlobalAveragePooling layer 주가
output = keras.layers.Dense(n_classes, activation="softmax")(avg) # 새로운 dense layer 추가
model = keras.Model(inputs=base_model.input, outputs=output)

83689472/83683744 [==============================] - 16s 0us/step


In [22]:
# 훈련 초기에는 사전훈련된 층의 가중치를 동결
for layer in base_model.layers:
    layer.trainable = False 

In [24]:
#최적화 함수 지정
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)

# 모델 컴파일
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# fitting
history = model.fit(train_set, epochs=5, validation_data=valid_set)

# 75~80% 수준에 머뭄


Epoch 1/5
86/86 [==============================] - 372s 4s/step - loss: 1.8903 - accuracy: 0.6843 - val_loss: 0.9163 - val_accuracy: 0.8621
Epoch 2/5
86/86 [==============================] - 361s 4s/step - loss: 0.5151 - accuracy: 0.9095 - val_loss: 0.7988 - val_accuracy: 0.8766
Epoch 3/5
86/86 [==============================] - 363s 4s/step - loss: 0.2725 - accuracy: 0.9352 - val_loss: 0.8340 - val_accuracy: 0.8639
Epoch 4/5
86/86 [==============================] - 409s 5s/step - loss: 0.1730 - accuracy: 0.9564 - val_loss: 0.6633 - val_accuracy: 0.8784
Epoch 5/5
86/86 [==============================] - 371s 4s/step - loss: 0.1113 - accuracy: 0.9656 - val_loss: 0.6618 - val_accuracy: 0.8693


In [26]:
# 동결 해제
for layer  in base_model.layers:
    layer.trainable=True

# 최적화함수
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

# 모델 컴파일
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# fitting
history = model.fit(train_set, epochs=5, validation_data=valid_set)

AttributeError: 'Functional' object has no attribute 'complie'